# Julia 实现

In [1]:
import Base:show, +, -, *, ==, zero, one, oneunit, rand

In [2]:
struct ℤ{N}
    buffer::Int
    N::Int
    value::Int
    function ℤ{N}(buffer::Int) where N
        value = buffer % N
        new{N}(buffer, N, value)
    end
end

show(io::IO, ::MIME"text/plain", n::ℤ{N}) where N = print(io, "$(n.value)(mod $(n.N))")
zero(::Type{ℤ{N}}) where N = ℤ{N}(0)
one(::Type{ℤ{N}}) where N = ℤ{N}(1)
zero(::ℤ{N}) where N = zero(ℤ{N})
oneunit(T::Type{ℤ{N}}) where N = one(T)
-(num::ℤ{N}) where N = ℤ{N}(N-num.value)
+(lhs::ℤ{N}, rhs::ℤ{N}) where N = ℤ{N}(lhs.value + rhs.value)
-(lhs::ℤ{N}, rhs::ℤ{N}) where N = lhs + (-rhs)
*(lhs::ℤ{N}, rhs::ℤ{N}) where N = ℤ{N}(lhs.value * rhs.value)
function rand(::Type{ℤ{N}}, dims::Tuple{Vararg{Int64, M}}) where {N, M}
    nums = [ℤ{N}(n) for n in 0:N-1]
    rand(nums, dims)
end

rand (generic function with 83 methods)

In [3]:
A = rand(ℤ{6}, (4_000, 2_000)); b = rand(ℤ{6}, (2_000,));

In [4]:
using BenchmarkTools
@benchmark $A*$b

BenchmarkTools.Trial: 206 samples with 1 evaluation.
 Range (min … max):  22.040 ms … 31.665 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     23.747 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   24.311 ms ±  1.389 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

             ██▅                                               
  ▂▃▁▁▁▁▁▂▄▃████▆▇█▄▄▅▄▃▄▂▄▅▄▃▂▃▃▁▁▁▁▃▁▃▁▁▃▁▁▂▃▁▁▁▂▁▂▂▂▁▁▃▁▁▂ ▃
  22 ms           Histogram: frequency by time        29.3 ms <

 Memory estimate: 93.80 KiB, allocs estimate: 2.

In [5]:
C = rand(4_000, 2_000); d = rand(2_000);

In [6]:
@benchmark $C*$d

BenchmarkTools.Trial: 3214 samples with 1 evaluation.
 Range (min … max):  1.256 ms …   4.713 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.456 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.537 ms ± 299.385 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ██▄                                                         
  ▆████▇▆▅▆▆▆▅▆▅▅▅▅▅▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▁▂▂▂ ▃
  1.26 ms         Histogram: frequency by time        2.61 ms <

 Memory estimate: 31.30 KiB, allocs estimate: 2.

# Python 实现

In [1]:
class Mod(object):
    def __init__(self, buffer, mod):
        assert(isinstance(buffer, int) and isinstance(mod, int))
        self.buffer = buffer
        self.mod = mod
        self.value = buffer % mod
    def __repr__(self):
        return f"{self.value}(mod {self.mod})"
    def __add__(self, rhs):
        assert(self.mod == rhs.mod)
        return Mod(self.value+rhs.value, self.mod)
    def __neg__(self):
        return Mod(self.mod-self.value, self.mod)
    def __sub__(self, rhs):
        return self + (-rhs)
    def __mul__(self, rhs):
        assert(self.mod == rhs.mod)
        return Mod(self.value*rhs.value, self.mod)

In [2]:
import numpy as np
from numpy.random import choice

In [3]:
def rand(mod, *args):
    nums = [Mod(n, mod) for n in range(mod)]
    return choice(nums, size=args)

In [4]:
A = rand(5, 4_000, 2_000); b = rand(5, 2_000)

In [5]:
%timeit A@b

4.55 s ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
C = np.random.rand(4_000, 2_000); d = np.random.rand(2_000)

In [7]:
%timeit C@d

1.52 ms ± 30.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
